In [23]:
%run model_sucrose.ipynb

Knocking out reactions mentioned in the paper by Wu et al (Wu et al., 2019):
- LDH which converts pyruvate to lactate
- PTA which converts Acetyl-CoA to Acetyl phosphate

### Finding knockout targets

#### Finding the LDH reaction:

In [24]:
model.reactions.query("LDH")

[<Reaction BETALDHx at 0x1f67da12ac0>,
 <Reaction LDH2 at 0x1f67dcedfd0>,
 <Reaction LDH_L at 0x1f67dd02a00>,
 <Reaction VALDHr at 0x1f67df80eb0>]

Checking if LDH_L is the one producing lactate

In [25]:
model.reactions.LDH_L.metabolites

{<Metabolite lac__L_c at 0x1f67d6399a0>: -1.0,
 <Metabolite nad_c at 0x1f67d63eb50>: -1.0,
 <Metabolite h_c at 0x1f67d62faf0>: 1.0,
 <Metabolite nadh_c at 0x1f67d63ed90>: 1.0,
 <Metabolite pyr_c at 0x1f67d6532b0>: 1.0}

In [26]:
model.metabolites.lac__L_c.name

'L-Lactate'

Yes. Seems like LDH_L is the reaction we're looking for.

#### Finding the PTA reaction

In [27]:
model.reactions.query("PTA")

[<Reaction APTA1i at 0x1f67d969c10>,
 <Reaction ASPTA at 0x1f67da12d00>,
 <Reaction GF6PTA at 0x1f67db6de20>,
 <Reaction PTAr at 0x1f67ddb2e80>,
 <Reaction PTA2 at 0x1f67ddb2880>]

Checking if PTAr is the reaction Acetyl phosphate:

In [28]:
model.reactions.PTAr.metabolites

{<Metabolite accoa_c at 0x1f67d5d96a0>: -1.0,
 <Metabolite pi_c at 0x1f67d639e20>: -1.0,
 <Metabolite actp_c at 0x1f67d5e2ca0>: 1.0,
 <Metabolite coa_c at 0x1f67d6008b0>: 1.0}

In [29]:
model.metabolites.actp_c.name

'Acetyl phosphate'

Yes. Seems like PTAr is the reaction we're looking for.

### Knocking out reactions

Now we'll try knocking it out each of the reactions and see what happens to the growth and surfactin yield:

Knockout of LDH_L:

In [30]:
with model:
    model.reactions.LDH_L.bounds = 0, 0
    print("LDH_L's bounds inside the with statement:")
    print(model.reactions.LDH_L.lower_bound, model.reactions.LDH_L.bounds)
    print('Mutant growth rate [mmol gDW^-1 h^-1]: ', model.optimize().objective_value)
    model.objective = model.reactions.DM_surfactin_c
    surfactin_production = model.optimize().objective_value
    print("Theoretical max. yield [mmol-succ / mmol-ac]:", surfactin_production / (-1*model.reactions.EX_glc__D_e.flux))

LDH_L's bounds inside the with statement:
0 (0, 0)
Mutant growth rate [mmol gDW^-1 h^-1]:  0.3436433767512154


ZeroDivisionError: float division by zero

Knockout of PTAr:

In [22]:
with model:
    model.reactions.PTAr.bounds = 0, 0
    print("LDH_L's bounds inside the with statement:")
    print(model.reactions.PTAr.lower_bound, model.reactions.PTAr.bounds)
    print('Mutant growth rate [mmol gDW^-1 h^-1]: ', model.optimize().objective_value)
    model.objective = model.reactions.DM_surfactin_c
    surfactin_production = model.optimize().objective_value
    print("Theoretical max. yield [mmol-succ / mmol-ac]:", surfactin_production / (-1*model.reactions.EX_glc__D_e.flux))

LDH_L's bounds inside the with statement:
0 (0, 0)
Mutant growth rate [mmol gDW^-1 h^-1]:  0.0743950621585464
Theoretical max. yield [mmol-succ / mmol-ac]: 0.043761801269733176


In both cases we see that the yield does not increase significantly